# Auto ML on Heart Failure Prediction Dataset

Importing required dependencies

In [1]:
import os
import joblib
import azureml.core
from azureml.core import Workspace, Experiment, Dataset, Environment
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.widgets import RunDetails
from azureml.train.automl import AutoMLConfig
from pprint import pprint # Used in printing automl model parameters
from azureml.core import Model # Used to get model information

# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

SDK version: 1.51.0


## Initializing the Workspace

Workspace initialization from the given configuration. 

In [2]:
ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

quick-starts-ws-252373
aml-quickstarts-252373
westus2
a0a76bad-11a1-4a2d-9887-97a29122c8ed


## Creating the Capstone experiment on Azure ML

The experiment to track all the submitted and completed runs in the given workspace.

In [3]:
# Choose a name for the run history container in the workspace
experiment_name = 'capstone-automl'
project_folder = './cpautoml-project'
experiment = Experiment(ws, experiment_name)

run = experiment.start_logging()

## Adding or Creating the Compute Cluster  

Creating the compute cluster for the AutoML run

In [4]:
# Choose a name for your cluster
# Compute name should contain only letters, digits, hyphen and should be 2-16 charachters long
#cluster_name = "aml-cluster"


compute_cluster_name = "notebook252373"

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

### YOUR CODE HERE ###

try:
    compute_target = ComputeTarget(workspace=ws, name=compute_cluster_name)
    print("Found existing cluster, use this cluster that was found.")
except:
    print("Creating new cluster...")
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_DS3_V2', max_nodes=4)
    compute_target = ComputeTarget.create(ws, compute_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True)
print(compute_target.get_status().serialize())

Found existing cluster, use this cluster that was found.

Running
{'errors': [], 'creationTime': '2024-02-10T23:01:44.392549+00:00', 'createdBy': {'userObjectId': '7a5bf30b-54a7-43d9-9728-1a42feb1a7bc', 'userTenantId': '660b3398-b80e-49d2-bc5b-ac1dc93b5254', 'userName': None}, 'modifiedTime': '2024-02-10T23:10:32.268816+00:00', 'state': 'Running', 'vmSize': 'Standard_DS3_v2'}


## Dataset Description

This project aims to leverage the power of Automated Machine Learning (AutoML) to predict mortality resulting from heart failure, a prevalent consequence of Cardiovascular diseases (CVDs). Heart failure manifests when the heart's ability to pump blood adequately to meet the body's metabolic demands diminishes, often leading to severe health complications and, in some cases, death.

The dataset chosen for this predictive task is the [Heart Failure Prediction dataset](https://raw.githubusercontent.com/robiulrafi/AZURE_ML_ND_PORTFOLIO/main/project_1/heart_failure_clinical_records_dataset.csv), which encompasses comprehensive clinical records from 299 heart failure patients. This dataset comprises a rich array of 12 features, capturing various aspects of patients' clinical conditions, physiological parameters, and lifestyle attributes.

Our primary objective revolves around training a robust binary classification model capable of predicting the DEATH_EVENT column. This binary indicator signifies whether a patient survived or succumbed to heart failure before the designated follow-up period elapsed. The predictive power of our model hinges on the information gleaned from the remaining 11 columns, serving as predictors. It's worth noting that the time feature, which may have provided temporal context, was deliberately excluded from the training process. This decision was made to ensure the model's practicality for future use, as obtaining time-based data for new patients post-deployment is often unfeasible.

The successful development and deployment of accurate prediction models hold immense potential for enhancing patient care and clinical decision-making within hospital settings. By enabling healthcare professionals to anticipate mortality risks associated with cardiovascular diseases more effectively, these models can facilitate timely interventions and personalized treatment strategies, ultimately improving patient outcomes and reducing the burden on healthcare systems.

In [5]:
from azureml.core.dataset import Dataset
ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')


found = False
key = "heart-failure-prediction"
description_text = "Prediction of patients’ survival with heart failure - Capstone project"

if key in ws.datasets.keys(): 
        found = True
        dataset = ws.datasets[key] 

if not found:
        # Create AML Dataset and register it into Workspace
        my_dataset = 'https://raw.githubusercontent.com/robiulrafi/AZURE_ML_ND_PORTFOLIO/main/project_1/heart_failure_clinical_records_dataset.csv'
        dataset = Dataset.Tabular.from_delimited_files(my_dataset)        
        # Register Dataset in Workspace
        dataset = dataset.register(workspace=ws,
                                   name=key,
                                   description=description_text)

df = dataset.to_pandas_dataframe()
df.describe()

quick-starts-ws-252373
aml-quickstarts-252373
westus2
a0a76bad-11a1-4a2d-9887-97a29122c8ed


,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time,DEATH_EVENT
count,299.000000,299.000000,299.000000,299.000000,299.000000,299.000000,299.000000,299.00000,299.000000,299.000000,299.00000,299.000000,299.00000
mean,60.833893,0.431438,581.839465,0.418060,38.083612,0.351171,263358.029264,1.39388,136.625418,0.648829,0.32107,130.260870,0.32107
std,11.894809,0.496107,970.287881,0.494067,11.834841,0.478136,97804.236869,1.03451,4.412477,0.478136,0.46767,77.614208,0.46767
min,40.000000,0.000000,23.000000,0.000000,14.000000,0.000000,25100.000000,0.50000,113.000000,0.000000,0.00000,4.000000,0.00000
25%,51.000000,0.000000,116.500000,0.000000,30.000000,0.000000,212500.000000,0.90000,134.000000,0.000000,0.00000,73.000000,0.00000
50%,60.000000,0.000000,250.000000,0.000000,38.000000,0.000000,262000.000000,1.10000,137.000000,1.000000,0.00000,115.000000,0.00000
75%,70.000000,1.000000,582.000000,1.000000,45.000000,1.000000,303500.000000,1.40000,140.000000,1.000000,1.00000,203.000000,1.00000
max,95.000000,1.000000,7861.000000,1.000000,80.000000,1.000000,850000.000000,9.40000,148.000000,1.000000,1.00000,285.000000,1.00000


In [6]:
# Review the dataset result
dataset.take(5).to_pandas_dataframe()

,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time,DEATH_EVENT
0,75.0,0,582,0,20,1,265000.00,1.9,130,1,0,4,1
1,55.0,0,7861,0,38,0,263358.03,1.1,136,1,0,6,1
2,65.0,0,146,0,20,0,162000.00,1.3,129,1,1,7,1
3,50.0,1,111,0,20,0,210000.00,1.9,137,1,0,7,1
4,65.0,1,160,1,20,0,327000.00,2.7,116,0,0,8,1


## AutoML Configuration

Following configuration is chosen:

1. `n_cross_validations`: 2
   - This parameter specifies the number of cross-validation folds to be used during training. Cross-validation is a technique used to assess the model's performance and generalization ability. A smaller number of folds (like 2 in this case) might be chosen for faster training, especially when computational resources or time are limited.

2. `primary_metric`: 'accuracy'
   - The primary metric determines what aspect of the model's performance to optimize during training. 'Accuracy' is a common metric for classification tasks, as it represents the proportion of correctly predicted instances out of the total instances. Maximizing accuracy ensures that the model makes accurate predictions overall.

3. `enable_early_stopping`: True
   - Enabling early stopping allows the training process to terminate if the performance metric stops improving. This helps prevent overfitting and saves computational resources by stopping training early if the model's performance has plateaued.

4. `max_concurrent_iterations`: 4
   - This parameter specifies the maximum number of parallel iterations or concurrent training jobs to execute. Setting it to 4 allows for multiple training iterations to be executed simultaneously, leveraging available computational resources efficiently and potentially reducing the overall training time.

5. `experiment_timeout_minutes`: 20
   - Sets the maximum amount of time, in minutes, allowed for the experiment to run. A value of 20 minutes ensures that the experiment doesn't run indefinitely and provides a time constraint for model training. This helps prevent excessive resource consumption and ensures timely completion of the experiment.

6. `verbosity`: logging.INFO
   - Controls the amount of logging information displayed during the training process. Setting it to `logging.INFO` ensures that informational messages are logged, providing insights into the training progress without overwhelming the output with excessive details.
   
The AutoMLConfig parameters are as follows:

1. `compute_target`:
   - Specifies the compute target where the training will be executed.

2. `task`:
   - Defines the type of task, in this case, 'classification' since we're dealing with a binary classification problem.

3. `training_data`:
   - Indicates the dataset to be used for training the model.

4. `label_column_name`:
   - Specifies the name of the target column ('DEATH_EVENT') that the model will predict.

5. `featurization`:
   - Controls the featurization process. Setting it to 'auto' allows AutoML to automatically handle feature engineering.

6. `debug_log`:
   - Specifies the file path to store debug logs generated during the training process.

7. `**automl_settings`:
   - Additional settings for the AutoML experiment, such as the number of cross-validations, primary metric, early stopping, maximum concurrent iterations, experiment timeout, and verbosity level. These settings are passed as keyword arguments to the AutoMLConfig.

In [7]:
# Put your automl settings here
automl_settings = {"n_cross_validations": 2,
                    "primary_metric": 'accuracy',
                    "enable_early_stopping": True,
                    "max_concurrent_iterations": 4,
                    "experiment_timeout_minutes": 20,
                    "verbosity": logging.INFO
                    }

# Parameters for AutoMLConfig

automl_config = AutoMLConfig(compute_target = compute_target,
                            task='classification',
                            training_data=dataset,
                            label_column_name='DEATH_EVENT',
                            featurization= 'auto',
                            debug_log = "automl_errors.log",
                            **automl_settings
                            )

In [8]:
# Submit your experiment
automl_run = experiment.submit(automl_config)

Submitting remote run.


Experiment,Id,Type,Status,Details Page,Docs Page
capstone-automl,AutoML_14165718-5495-4815-b083-0313873ebed2,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation


## Run Details

In [9]:
RunDetails(automl_run).show()
automl_run.wait_for_completion(show_output=True)

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

Experiment,Id,Type,Status,Details Page,Docs Page
capstone-automl,AutoML_14165718-5495-4815-b083-0313873ebed2,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation



Current status: FeaturesGeneration. Generating features for the dataset.
Current status: ModelSelection. Beginning model selection.

********************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData

********************************************************************************************

TYPE:         Missing feature values imputation
STATUS:       PASSED
DESCRIPTION:  No feature missing values were detected in the training data.
              Learn more about missing value imputation: https://aka.ms/AutomatedMLFeaturization

********************************************************************************************

TYPE:         High cardinality feature detection
STATUS:       PASSED
DESCRIPTI

{'runId': 'AutoML_14165718-5495-4815-b083-0313873ebed2',
 'target': 'notebook252373',
 'status': 'Completed',
 'startTimeUtc': '2024-02-11T03:00:07.428536Z',
 'endTimeUtc': '2024-02-11T03:15:30.053241Z',
 'services': {},
 'warnings': [{'source': 'JasmineService',
   'message': 'No scores improved over last 10 iterations, so experiment stopped early. This early stopping behavior can be disabled by setting enable_early_stopping = False in AutoMLConfig for notebook/python SDK runs.'}],
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'accuracy',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': '2',
  'target': 'notebook252373',
  'AMLSettingsJsonString': '{"path":null,"name":"capstone-automl","subscription_id":"a0a76bad-11a1-4a2d-9887-97a29122c8ed","resource_group":"aml-quickstarts-252373","workspace_name":"quick-starts-ws-252373","region":"westus2","compute_target":"notebook252373"

## Best Model

In [10]:
# Get best run and model
best_run_automl, best_model_automl = automl_run.get_output()

Package:azureml-automl-runtime, training version:1.52.0.post1, current version:1.51.0.post1
Package:azureml-core, training version:1.52.0, current version:1.51.0
Package:azureml-dataprep, training version:4.11.4, current version:4.10.8
Package:azureml-dataprep-rslex, training version:2.18.4, current version:2.17.12
Package:azureml-dataset-runtime, training version:1.52.0, current version:1.51.0
Package:azureml-defaults, training version:1.52.0, current version:1.51.0
Package:azureml-interpret, training version:1.52.0, current version:1.51.0
Package:azureml-mlflow, training version:1.52.0, current version:1.51.0
Package:azureml-pipeline-core, training version:1.52.0, current version:1.51.0
Package:azureml-responsibleai, training version:1.52.0, current version:1.51.0
Package:azureml-telemetry, training version:1.52.0, current version:1.51.0
Package:azureml-train-automl-client, training version:1.52.0, current version:1.51.0.post1
Package:azureml-train-automl-runtime, training version:1.

In [11]:
best_run_automl

Experiment,Id,Type,Status,Details Page,Docs Page
capstone-automl,AutoML_14165718-5495-4815-b083-0313873ebed2_39,azureml.scriptrun,Completed,Link to Azure Machine Learning studio,Link to Documentation


In [12]:
print('Best Run Id: ' + best_run_automl.id,
     'Best Model Name: ' + best_run_automl.properties['model_name'])

Best Run Id: AutoML_14165718-5495-4815-b083-0313873ebed2_39 Best Model Name: AutoML14165718539


In [13]:
# get_metrics()
# Returns the metrics
print("Best run metrics :",best_run_automl.get_metrics())

Best run metrics : {'weighted_accuracy': 0.8855926584916601, 'norm_macro_recall': 0.6671838737601634, 'AUC_weighted': 0.8991608440300494, 'AUC_macro': 0.8991608440300494, 'precision_score_micro': 0.862841163310962, 'recall_score_weighted': 0.862841163310962, 'precision_score_weighted': 0.8638818146538663, 'AUC_micro': 0.912656872313059, 'accuracy': 0.862841163310962, 'precision_score_macro': 0.8507073283858998, 'recall_score_macro': 0.8335919368800817, 'balanced_accuracy': 0.8335919368800817, 'average_precision_score_weighted': 0.9024786847088246, 'matthews_correlation': 0.683670083486865, 'f1_score_macro': 0.8395979757029699, 'f1_score_weighted': 0.8613931698914903, 'average_precision_score_micro': 0.9115423962232916, 'average_precision_score_macro': 0.8777450983857102, 'log_loss': 0.38544209965433607, 'f1_score_micro': 0.8628411633109618, 'recall_score_micro': 0.862841163310962, 'confusion_matrix': 'aml://artifactId/ExperimentRun/dcid.AutoML_14165718-5495-4815-b083-0313873ebed2_39/co

In [14]:
# get_details()
# Returns a dictionary with the details for the run
print("Best run details :",best_run_automl.get_details())

Best run details : {'runId': 'AutoML_14165718-5495-4815-b083-0313873ebed2_39', 'target': 'notebook252373', 'status': 'Completed', 'startTimeUtc': '2024-02-11T03:14:16.314572Z', 'endTimeUtc': '2024-02-11T03:15:09.979514Z', 'services': {}, 'properties': {'runTemplate': 'automl_child', 'pipeline_id': '__AutoML_Ensemble__', 'pipeline_spec': '{"pipeline_id":"__AutoML_Ensemble__","objects":[{"module":"azureml.train.automl.ensemble","class_name":"Ensemble","spec_class":"sklearn","param_args":[],"param_kwargs":{"automl_settings":"{\'task_type\':\'classification\',\'primary_metric\':\'accuracy\',\'verbosity\':20,\'ensemble_iterations\':15,\'is_timeseries\':False,\'name\':\'capstone-automl\',\'compute_target\':\'notebook252373\',\'subscription_id\':\'a0a76bad-11a1-4a2d-9887-97a29122c8ed\',\'region\':\'westus2\',\'spark_service\':None}","ensemble_run_id":"AutoML_14165718-5495-4815-b083-0313873ebed2_39","experiment_name":"capstone-automl","workspace_name":"quick-starts-ws-252373","subscription_id"

In [15]:
# get_properties()
# Fetch the latest properties of the run from the service
print("Best run properties :",best_run_automl.get_properties())

Best run properties : {'runTemplate': 'automl_child', 'pipeline_id': '__AutoML_Ensemble__', 'pipeline_spec': '{"pipeline_id":"__AutoML_Ensemble__","objects":[{"module":"azureml.train.automl.ensemble","class_name":"Ensemble","spec_class":"sklearn","param_args":[],"param_kwargs":{"automl_settings":"{\'task_type\':\'classification\',\'primary_metric\':\'accuracy\',\'verbosity\':20,\'ensemble_iterations\':15,\'is_timeseries\':False,\'name\':\'capstone-automl\',\'compute_target\':\'notebook252373\',\'subscription_id\':\'a0a76bad-11a1-4a2d-9887-97a29122c8ed\',\'region\':\'westus2\',\'spark_service\':None}","ensemble_run_id":"AutoML_14165718-5495-4815-b083-0313873ebed2_39","experiment_name":"capstone-automl","workspace_name":"quick-starts-ws-252373","subscription_id":"a0a76bad-11a1-4a2d-9887-97a29122c8ed","resource_group_name":"aml-quickstarts-252373"}}]}', 'training_percent': '100', 'predicted_cost': None, 'iteration': '39', '_aml_system_scenario_identification': 'Remote.Child', '_azureml.Co

In [16]:
best_run_automl.get_tags()

{'_aml_system_azureml.automlComponent': 'AutoML',
 '_aml_system_ComputeTargetStatus': '{"AllocationState":"steady","PreparingNodeCount":0,"RunningNodeCount":0,"CurrentNodeCount":1}',
 'mlflow.source.type': 'JOB',
 'mlflow.source.name': 'automl_driver.py',
 '_aml_system_codegen': 'completed',
 '_aml_system_automl_is_child_run_end_telemetry_event_logged': 'True'}

## Model Deployment

In [18]:
# Register the model
best_autoML_Model=best_run_automl.register_model(model_path='outputs/', model_name='automl-best-model',
                   tags={'Training context':'AutoML', 'type': 'Classification'},
                  
                   description = 'AutoML based Heart Failure Predictor')

In [19]:
# List registered models to verify if model has been saved
for model in Model.list(ws):
    print(model.name, 'version:', model.version)
    for tag_name in model.tags:
        tag = model.tags[tag_name]
        print ('\t',tag_name, ':', tag)
    for prop_name in model.properties:
        prop = model.properties[prop_name]
        print ('\t',prop_name, ':', prop)
    print('\n')

automl-best-model version: 1
	 Training context : AutoML
	 type : Classification


hyperdrive-best-model version: 1
	 Training context : Parameterized SKLearn Estimator
	 type : Classification
	 Accuracy : 0.75




In [49]:
best_run_automl.get_file_names()

# Download the yaml file that includes the environment dependencies
best_run_automl.download_file('outputs/conda_env_v_1_0_0.yml', 'env.yml')

In [51]:
# Download the model file

best_run_automl.download_file('outputs/model.pkl', 'Automl_model.pkl')

In [53]:
best_run_automl.register_model(model_name = "best_run_automl.pkl", model_path = './outputs/')

print(best_run_automl)

Run(Experiment: capstone-automl,
Id: AutoML_14165718-5495-4815-b083-0313873ebed2_39,
Type: azureml.scriptrun,
Status: Completed)


In [27]:
best_run_automl.download_file('outputs/model.pkl', './model.pkl')

# Download scoring file
best_run_automl.download_file('outputs/scoring_file_v_1_0_0.py', './score.py')

best_run_automl.download_file('outputs/conda_env_v_1_0_0.yml', 'envFile.yml')

In [58]:
model = automl_run.register_model(model_name = 'best_run_automl.pkl')
print(automl_run.model_id)

# https://knowledge.udacity.com/questions/463620

environment = best_run_automl.get_environment()
entry_script='inference/scoring.py'
best_run_automl.download_file('outputs/scoring_file_v_1_0_0.py', entry_script)


inference_config = InferenceConfig(entry_script = entry_script, environment = environment)

# Deploying the model via ACI WebService
# https://github.com/MicrosoftDocs/azure-docs/blob/master/articles/machine-learning/how-to-deploy-azure-container-instance.md

deployment_config = AciWebservice.deploy_configuration(cpu_cores = 1, 
                                                    memory_gb = 1, 
                                                    auth_enabled= True, 
                                                    enable_app_insights= True)

service = Model.deploy(ws, "deployservice", [model], inference_config, deployment_config)
service.wait_for_deployment(show_output = True)

best_run_automl.pkl
Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2024-02-11 04:56:35+00:00 Creating Container Registry if not exists.
2024-02-11 04:56:36+00:00 Registering the environment.
2024-02-11 04:56:36+00:00 Use the existing image.
2024-02-11 04:56:36+00:00 Submitting deployment to compute.
2024-02-11 04:56:43+00:00 Checking the status of deployment deployservice..
2024-02-11 05:00:19+00:00 Checking the status of inference endpoint deployservice.
Succeeded
ACI service creation operation finished, operation "Succeeded"


In [60]:
import json
import requests

# import test data
test_df = df.sample(5) # data is the pandas dataframe of the original data
label_df = test_df.pop('DEATH_EVENT')

test_sample = json.dumps({'data': test_df.to_dict(orient='records')})
print(test_sample)

{"data": [{"age": 45.0, "anaemia": 0, "creatinine_phosphokinase": 582, "diabetes": 1, "ejection_fraction": 38, "high_blood_pressure": 1, "platelets": 263358.03, "serum_creatinine": 1.18, "serum_sodium": 137, "sex": 0, "smoking": 0, "time": 185}, {"age": 52.0, "anaemia": 0, "creatinine_phosphokinase": 132, "diabetes": 0, "ejection_fraction": 30, "high_blood_pressure": 0, "platelets": 218000.0, "serum_creatinine": 0.7, "serum_sodium": 136, "sex": 1, "smoking": 1, "time": 112}, {"age": 46.0, "anaemia": 0, "creatinine_phosphokinase": 168, "diabetes": 1, "ejection_fraction": 17, "high_blood_pressure": 1, "platelets": 271000.0, "serum_creatinine": 2.1, "serum_sodium": 124, "sex": 0, "smoking": 0, "time": 100}, {"age": 65.0, "anaemia": 1, "creatinine_phosphokinase": 59, "diabetes": 1, "ejection_fraction": 60, "high_blood_pressure": 0, "platelets": 172000.0, "serum_creatinine": 0.9, "serum_sodium": 137, "sex": 0, "smoking": 0, "time": 107}, {"age": 66.0, "anaemia": 1, "creatinine_phosphokinase

In [61]:
# Authentication is enabled, so I use the get_keys method to retrieve the primary and secondary authentication keys:
primary, secondary = service.get_keys()

print('Service state: ' + service.state)
print('Service scoring URI: ' + service.scoring_uri)
print('Service Swagger URI: ' + service.swagger_uri)
print('Service primary authentication key: ' + primary)

Service state: Healthy
Service scoring URI: http://7594a79a-499d-44b8-a032-fa371775088f.westus2.azurecontainer.io/score
Service Swagger URI: http://7594a79a-499d-44b8-a032-fa371775088f.westus2.azurecontainer.io/swagger.json
Service primary authentication key: TAT3auUpdVqkKNsipguvsblYGxHG3H4E


In [47]:
test_sample

'{"data": [{"age": 60.0, "anaemia": 0, "creatinine_phosphokinase": 59, "diabetes": 0, "ejection_fraction": 25, "high_blood_pressure": 1, "platelets": 212000.0, "serum_creatinine": 3.5, "serum_sodium": 136, "sex": 1, "smoking": 1, "time": 187}, {"age": 60.0, "anaemia": 0, "creatinine_phosphokinase": 582, "diabetes": 1, "ejection_fraction": 38, "high_blood_pressure": 1, "platelets": 451000.0, "serum_creatinine": 0.6, "serum_sodium": 138, "sex": 1, "smoking": 1, "time": 40}, {"age": 70.0, "anaemia": 0, "creatinine_phosphokinase": 571, "diabetes": 1, "ejection_fraction": 45, "high_blood_pressure": 1, "platelets": 185000.0, "serum_creatinine": 1.2, "serum_sodium": 139, "sex": 1, "smoking": 1, "time": 33}, {"age": 60.0, "anaemia": 1, "creatinine_phosphokinase": 76, "diabetes": 1, "ejection_fraction": 25, "high_blood_pressure": 0, "platelets": 196000.0, "serum_creatinine": 2.5, "serum_sodium": 132, "sex": 0, "smoking": 0, "time": 77}, {"age": 40.0, "anaemia": 0, "creatinine_phosphokinase": 47

In [66]:
# predict using the deployed model
result = service.run(test_sample)
print(result)

{"result": [0, 0, 1, 0, 0]}


In [71]:
## Endpoint Consumption
import requests
import json

# URL for the web service, should be similar to:
# 'http://8530a665-66f3-49c8-a953-b82a2d312917.eastus.azurecontainer.io/score'

scoring_uri = service.scoring_uri
# If the service is authenticated, set the key or token
key = primary

# Two sets of data to score, so we get two results back
data = {"data":
        [
          {
           "age": 60, 
           "anaemia": 0, 
           "creatinine_phosphokinase": 59, 
           "diabetes": 0, 
           "ejection_fraction": 25, 
           "high_blood_pressure": 1, 
           "platelets": 212000, 
           "serum_creatinine": 3.5, 
           "serum_sodium": 136, 
           "sex": 1, 
           "smoking": 1,
           "time": 187
          },
          {
           "age": 51, 
           "anaemia": 0, 
           "creatinine_phosphokinase": 1380, 
           "diabetes": 0, 
           "ejection_fraction": 25, 
           "high_blood_pressure": 1, 
           "platelets": 271000, 
           "serum_creatinine": 0.9, 
           "serum_sodium": 130, 
           "sex": 1, 
           "smoking": 0,
           "time": 38
          },
      ]
    }
# Convert to JSON string
input_data = json.dumps(data)
with open("data.json", "w") as _f:
    _f.write(input_data)

# Set the content type
headers = {'Content-Type': 'application/json'}
# If authentication is enabled, set the authorization header
headers['Authorization'] = f'Bearer {key}'

# Make the request and display the response
resp = requests.post(scoring_uri, input_data, headers=headers)
print(resp.json())
print("++++++++++++++++++++++++++++++")
print("Expected result: [false, true], where 'true' means '1' as result in the 'DEATH_EVENT' column")

{"result": [0, 1]}
++++++++++++++++++++++++++++++
Expected result: [false, true], where 'true' means '1' as result in the 'DEATH_EVENT' column


In [69]:
## Printing The Service Logs
print(service.get_logs())

2024-02-11T04:59:59,641069000+00:00 - rsyslog/run 
2024-02-11T04:59:59,653983800+00:00 - gunicorn/run 
2024-02-11T04:59:59,659327000+00:00 | gunicorn/run | 
2024-02-11T04:59:59,669174400+00:00 | gunicorn/run | ###############################################
2024-02-11T04:59:59,672412800+00:00 - nginx/run 
2024-02-11T04:59:59,675255100+00:00 | gunicorn/run | AzureML Container Runtime Information
2024-02-11T04:59:59,682952500+00:00 | gunicorn/run | ###############################################
2024-02-11T04:59:59,685503000+00:00 | gunicorn/run | 
2024-02-11T04:59:59,693802400+00:00 | gunicorn/run | 
2024-02-11T04:59:59,719037700+00:00 | gunicorn/run | AzureML image information: openmpi4.1.0-ubuntu20.04, Materializaton Build:20230628.v2
2024-02-11T04:59:59,730974300+00:00 | gunicorn/run | 
2024-02-11T04:59:59,732852400+00:00 | gunicorn/run | 
2024-02-11T04:59:59,742234400+00:00 | gunicorn/run | PATH environment variable: /azureml-envs/azureml-automl/bin:/opt/miniconda/bin:/usr/local/sbi

In [ ]:
service.delete()